<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generator

**Description:** This Notebook read a referential, take a notebook template, and create as much notebook as rows on the referentials. 

## Input

In [ ]:
pip install nbformat

## Model

In [ ]:
import nbformat as nbf
import copy
import pandas as pd

# Read the CSV file
df = pd.read_csv('../inputs/referential.csv')

# Start the pipeline string
pipeline_string = ''

# Loop over the rows in the DataFrame
for i, row in df.iterrows():
    # Each row in the DataFrame becomes a dictionary
    replacement = row.to_dict()

    # Read the original notebook
    with open('YahooFinance_Template_Get_[ENTITY]_trends_and_predictions.ipynb') as f:
        nb = nbf.read(f, as_version=4)

    # Make a deep copy of the notebook object
    new_nb = copy.deepcopy(nb)

    # Loop over the cells in the notebook
    for cell in new_nb.cells:
        # Only modify code cells
        if cell.cell_type == 'code':
            # Replace the NAME_INPUT, TICKER_INPUT, and RELATIVE_INDEX values
            for key, value in replacement.items():
                cell.source = cell.source.replace(key, str(value))

    # Write the new notebook to a file
    # Include the NAME_INPUT value in the filename
    notebook_name = f'YahooFinance_Get_{replacement["NAME_INPUT"]}_trends_and_predictions.ipynb'
    with open(notebook_name, 'w') as f:
        nbf.write(new_nb, f)

    # Add the notebook to the pipeline string
    pipeline_string += f'NotebookStep("{replacement["NAME_INPUT"]}", "{notebook_name}") >> \\\n'

# Read the pipeline notebook
with open('__pipeline__.ipynb') as f:
    pipeline_nb = nbf.read(f, as_version=4)

# Find the cell that contains 'FINANCE_INDICATORS'
for cell in pipeline_nb.cells:
    if 'FINANCE_INDICATORS' in cell.source:
        # Replace 'FINANCE_INDICATORS' with the new pipeline string
        cell.source = cell.source.replace('FINANCE_INDICATORS', pipeline_string)
        break

# Write the modified pipeline notebook back to the file
with open('__pipeline__.ipynb', 'w') as f:
    nbf.write(pipeline_nb, f)
